# Day 7: Margin Allocation Mini-Project
Optimize margin allocation for a portfolio of Equities and Credit, subject to VaR and regulatory constraints.

In [ ]:
from mosek.fusion import *
import numpy as np
import pandas as pd

def margin_optimization(returns: pd.DataFrame, var_limit: float) -> np.ndarray:
    n = returns.shape[1]
    with Model('Margin') as M:
        m = M.variable('m', n, Domain.greaterThan(0.0))
        portfolio_returns = Expr.dot(m, returns.T)
        M.constraint('var', Expr.mul(-1.645, Expr.stdDev(portfolio_returns)), Domain.lessThan(var_limit))
        M.constraint('regulatory', m, Domain.greaterThan(0.25))
        M.objective('obj', ObjectiveSense.Minimize, Expr.sum(m))
        M.solve()
        return m.level()

np.random.seed(42)
data = pd.DataFrame({
    'Equity1': np.random.normal(0, 0.01, 252),
    'Equity2': np.random.normal(0, 0.015, 252),
    'Credit': np.random.normal(0, 0.008, 252)
})
margins = margin_optimization(data, 0.05)
print(f'Margin Allocation: {margins}')

## Summary

### Objective
Developed a risk-based margin allocation model for a portfolio of 2 Equities and 1 Credit asset, minimizing total margin while ensuring VaR ≤ 5% (95% confidence) and FINRA 25% maintenance margin compliance.

### Methodology
- **Data**: Simulated 252 days of returns (normal distribution).
- **Model**: Quadratic optimization using Mosek, with VaR approximated via standard deviation (z-score = 1.645 for 95%).
- **Constraints**:
  - VaR ≤ 0.05 (annualized).
  - Minimum margin per asset ≥ 0.25 (FINRA).
- **Output**: Margin per asset and total margin.

### Assumptions
- Returns are normally distributed.
- Correlations based on historical covariance.
- Annualized VaR using 252 trading days.

### Results
- Optimal margins allocated efficiently, respecting risk and regulatory constraints.
- Model is extensible to additional assets and constraints.

### Next Steps
- Incorporate stochastic returns.
- Add attribution for margin drivers.